#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [2]:
print("Before starting the entire loop. First Check what is the maximum size of the rock you can run the code for...")

Before starting the entire loop. First Check what is the maximum size of the rock you can run the code for...


#### Automate the GLSZM feature extraction process

In [45]:
%%time
print("Starting the automation loop for GLSZM feature extraction")

# All the greyscale images of various porous media types are named as data#.nrrd. 
# Iterate over the names to calculate the GLSZM features
for num_img in range(1, 2):
    
    # Create the variables for image and label
    global image_1, label_1

    # Path of the images used in REV analysis
    path ='D:\\Ankita PhD\\pyradiomics\\data\\data{}.nrrd'.format(num_img)
    
    # Read the nrrd image using the SITK library. Nrrd images are 8-bit images with intensity values between 0 to 255
    image_1 = sitk.ReadImage(path) 
    
    # Set the settings paramter needed for the bin_width and label
    # Bin_Width = 1 (as I don't want any further reduction in greyscale values and want to preserve the information)
    # label is a nrrd image that incorporates the window size and hence, gives a ROI for the GLSZM calculation.
    # The minimum size is set by the user and the mavimum size is the size of the image
    settings = {}
    settings['label'] = 255
    settings['binWidth'] = 1

    # Set-up logging for all the calculations
    log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\log{}.txt'.format(num_img)
    handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
    handler.setFormatter(formatter)
    radiomics.logger.addHandler(handler)

    # Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
    # Verbositiy level, the logger level will also determine the amount of information printed to the output
    radiomics.logger.setLevel(logging.DEBUG)

    # the size of the mask (same as label) should be the same as that of the image
    mask_x = image_1.GetWidth()
    mask_y = image_1.GetWidth()
    mask_z = image_1.GetWidth()

    #list for all the features being extracted
    v1 = []
    v2 = []
    v3 = []
    v4 = []
    v5 = []
    v6 = []
    v7 = []
    v8 = []
    v9 = []
    v10 = []
    v11 = []
    v12 = []
    v13 = []
    v14 = []
    v15 = []
    v16 = []
    v1_norm = []
    v3_norm = []
    v4_norm = []
    v5_norm = []
    v6_norm = []
    v7_norm = []
    v9_norm = []
    v12_norm = []
    v16_norm = []

    print("Starting the domain centre calculcation for image",str(num_img))
    
    # here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
    # sizes increase with every iteration
    u_window_size = [2, 3, 4, 5, 6, 7, 8 , 9, 10, 20, 30, 40, 50, 60, 90, 100, 120, 150, 180, 200]#, 250, 275, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800] #200, 150, 100, 60, 50, 40, 30, 20, 10]# 650, 600, 550, 500, 450, 400, 350, 300, 250, 200, 150, 100, 60, 50, 40, 30, 20, 10] #10, 20, 30, 40, 50, 60, 100, 150, 200, 250, 300, 350, 400, 450, 600, 650]
    global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
    
    for window_size in u_window_size:  

        i_loc = int((mask_x - window_size)/2)
        j_loc = int((mask_y - window_size)/2)
        k_loc = int((mask_z - window_size)/2)

        # Generate the mask
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc+window_size,j_loc:j_loc+window_size,k_loc:k_loc+window_size] = 255
        #print("Window Location", str(i_loc), str(i_loc+window_size), str(j_loc), str(j_loc+window_size), str(k_loc), str(k_loc+window_size))
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        label_1.CopyInformation(image_1)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        print("Cropped Mask and Image created for domain centre size =",str(window_size))
        
        import radiomics
        from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
        #calculate the original glszm
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        result = glszmFeatures.execute()
        
        print("Calculated GLSZM.....")
        
        glszm_org = glszmFeatures.P_glszm
        #The graylevels in original ROI after discretization
        graylevels_org = glszmFeatures.coefficients['grayLevels']
        print('1')
        #vector of all the gray-level valus present in the original ROI
        iv_org = glszmFeatures.coefficients['ivector']
        print('2')
        #vector of all the size zones in the original GLSZM
        jv_org = glszmFeatures.coefficients['jvector']
        
        # sum of all the size zones for all the graylevels of a particular size zones
        # for every column of GLSZM, add all the rows
        ps_org = glszmFeatures.coefficients['ps']
        
        #Remove the mineral or saturated phase if it exists FROM GLSZM
        if iv_org[iv_org.size - 1] == 256:
            print('Mineral phase is present because the graylevel of', iv_org[iv_org.size-1], 'exists.')
            mineralGrayLevels_idx = iv_org.size-1
            #Calculate the new GLSZM
            glszm_no_mineral = np.delete(glszm_org, mineralGrayLevels_idx, 1)
            s_nm = glszm_no_mineral.shape
            
            #Calculate the new ivector
            iv_new = iv_org[0:iv_org.size-1]  # shape (Ng,)
            
            #Calculate the new ps array with zero values
            glszm_2d_nm = glszm_no_mineral[0, 0:s_nm[1], 0:s_nm[2]]
            ps_new_zeros = np.sum(glszm_2d_nm, 0)
            ps_reshape = np.reshape(ps_new_zeros, (1,s_nm[2]))

            #Delete columns that specify zone sizes not present in the ROI
            emptyZoneSizes = np.where(np.sum(ps_reshape, 0) == 0)
            glszm_new = np.delete(glszm_no_mineral, emptyZoneSizes, 2)
            s_new = glszm_new.shape
            jv_new = np.delete(jv_org, emptyZoneSizes)
            ps_new = np.delete(ps_org, emptyZoneSizes, 1)
            
            # Get the number of zones in this GLSZM
            Nz_new = np.sum(glszm_new, (1, 2))  # shape (Nvox,)
            Nz_new[Nz_new == 0] = 1  # set sum to numpy.spacing(1) if sum is 0?

            # Get the number of voxels represented by this GLSZM: Multiply the zones by their size and sum them
            Np_new = np.sum(ps_new * jv_new[None, :], 1)  # shape (Nvox, )
            Np_new[Np_new == 0] = 1
            
            # Get the pg
            glszm_2d_new = glszm_new[0, 0:s_new[1], 0:s_new[2]]
            pg_reshape = np.sum(glszm_2d_new, 1)
            pg_new = np.reshape(pg_reshape, (1,s_new[1]))
            
        else:
            
            #just crab the coeffiecients as usual
            iv_new = iv_org
            jv_new = jv_org
            glszm_new = glszm_org
            Nz_new = glszmFeatures.coefficients['Nz']
            Np_new = glszmFeatures.coefficients['Np']
            pg_new = glszmFeatures.coefficients['pg']
            ps_new = glszmFeatures.coefficients['ps']
                    
        jvector = jv_new
        ivector = iv_new
        ps = ps_new
        pg = pg_new
        Np = Np_new
        Nz = Nz_new
        pg_norm = pg/Nz[:, None]
        ps_norm = ps/Nz[:, None]
        eps = np.spacing(1)
        glszm = glszm_new
        print('The shape of the old glszm is', str(glszm_org.shape), '. The shape of the new glszm is', str(glszm.shape))
        glszm_norm = glszm_new/Nz[:, None, None]
        
        print("GLSZM processing complete ...")
        
        #del glszm_org, glszm_new, iv_org, jv_org, ps_org, mineralGrayLevels_idx, glszm_no_mineral, s_nm, iv_new, glszm_2d_nm
        #del ps_new_zeros, ps_reshape, emptyZoneSizes, glszm_new, s_new, jv_new, ps_new, Nz_new, Np_new, glszm_2d_nm, pg_reshape
        #del pg_new
            
        #Calculate glszm features and normalize the features
        print("Initiating GLSZM feature calculation...... ")
        lae =[]
        
        sae = np.sum(ps / (jvector[None, :] ** 2), 1) / Nz #small area emphaisis
        sae = sae.astype(float)
        lae = np.sum(ps * (jvector[None, :] ** 2), 1) / Nz #large area emphaisis
        iv = np.sum(pg ** 2, 1) / Nz #graylevel non-uniformity
        ivn = np.sum(pg ** 2, 1) / Nz ** 2 #graylevel non-uniformity normalized
        szv = np.sum(ps ** 2, 1) / Nz #size zone uniformity
        szvn = np.sum(ps ** 2, 1) / Nz ** 2 #size zone uniformity normalised
        zp = Nz / Np # zone percentage
        u_i = np.sum(pg_norm * ivector[None, :], 1, keepdims=True) # mean calculation for graylevel variance
        glv = np.sum(pg_norm * (ivector[None, :] - u_i) ** 2, 1) # graylevel variance
        u_j = np.sum(ps_norm * jvector[None, :], 1, keepdims=True) #mean caluclation of zone variance
        zv = np.sum(ps_norm * (jvector[None, :] - u_j) ** 2, 1) #zone variance
        ze = -np.sum(glszm_norm * np.log2(glszm_norm + eps), (1, 2)) #zone entropy
        lie = np.sum(pg / (ivector[None, :] ** 2), 1) / Nz #Low graylevel zone emphaisis
        hie = np.sum(pg * (ivector[None, :] ** 2), 1) / Nz #high graylevel zone emphaisis
        lisae = np.sum(glszm / ((ivector[None, :, None] ** 2) * (jvector[None, None, :] ** 2)), (1, 2)) / Nz #small area low graylevel emphaisis
        hisae = np.sum(glszm * (ivector[None, :, None] ** 2) / (jvector[None, None, :] ** 2), (1, 2)) / Nz #small area high graylevel emphaisis
        lilae = np.sum(glszm * (jvector[None, None, :] ** 2) / (ivector[None, :, None] ** 2), (1, 2)) / Nz #large area low gray level emphaisis
        hilae = np.sum(glszm * (ivector[None, :, None] ** 2) * (jvector[None, None, :] ** 2), (1, 2)) / Nz #large area high gray level emphaisis
        
        #normalise features by the number of voxels
        iv_np = iv/Np
        glv_np = glv/Np
        hie_np = hie/Np
        lae_np = lae/Np
        hilae_np = hilae/Np
        lilae_np = lilae/Np
        szv_np = szv/Np
        hisae_np = hisae/Np
        zv_np = zv/Np
        
        #append values
        v1.append(iv)
        v1_norm.append(iv_np)
        v2.append(ivn)
        v3.append(glv)
        v3_norm.append(glv_np)
        v4.append(hie)
        v4_norm.append(hie_np)
        v5.append(lae)
        v5_norm.append(lae_np)
        v6.append(hilae)
        v6_norm.append(hilae_np)
        v7.append(lilae)
        v7_norm.append(lilae_np)
        v8.append(lie)
        v9.append(szv)
        v9_norm.append(szv_np)
        v10.append(szvn)
        v11.append(sae)
        v12.append(hisae)
        v12_norm.append(hisae_np)
        v13.append(lisae)
        v14.append(ze)
        v15.append(zp)
        v16.append(zv)
        v16_norm.append(zv_np)
        
        #del sae, lae, iv, ivn, szv, szvn, zp, u_i, glv, u_j, zv, ze, lie, hie, lisae, hisae, lilae, hilae
        #del iv_np, glv_np, hie_np, lae_np, hilae_np, lilae_np, szv_np, hisae_np, zv_np
        
        
        print("Completed calculations for domain centre size =", str(window_size))
    
    # Create a data-frame to store all the GLSZM features for the image    
    my_df = pd.DataFrame(list(zip(u_window_size, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, \
                                  v1_norm, v3_norm, v4_norm, v5_norm, v6_norm, v7_norm, v9_norm, v12_norm, v16_norm)))
    my_df = my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance", 17: "Norm GrayLevelNonUniformity", \
                                   18: "Norm GrayLevelVariance", 19: "Norm HighGrayLevelZoneEmphasis", 20: "Norm LargeAreaEmphasis", \
                                   21: "Norm LargeAreaHighGrayLevelEmphasis", 22: "Norm LargeAreaLowGrayLevelEmphasis", \
                                   23: "Norm SizeZoneNonUniformity", 24:"Norm SmallAreaHighGrayLevelEmphasis", 25: "Norm ZoneVariance"})    
    
    name_csv = 'glszm_crrtd_dc_data{}.csv'.format(num_img)
    my_df.to_csv(name_csv, index=False, header=True)
    
    print("Stored all the GLSZM features for image", str(num_img), "in a csv file")
    
    #free the memory and delete all the variables
    del image_1
    del my_df
    del glszmFeatures

Starting the automation loop for GLSZM feature extraction
Starting the domain centre calculcation for image 1
Cropped Mask and Image created for domain centre size = 2
Calculated GLSZM.....
1
2
The shape of the old glszm is (1, 7, 2) . The shape of the new glszm is (1, 7, 2)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
Completed calculations for domain centre size = 2
Cropped Mask and Image created for domain centre size = 3
Calculated GLSZM.....
1
2
The shape of the old glszm is (1, 12, 4) . The shape of the new glszm is (1, 12, 4)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
Completed calculations for domain centre size = 3
Cropped Mask and Image created for domain centre size = 4
Calculated GLSZM.....
1
2
The shape of the old glszm is (1, 16, 5) . The shape of the new glszm is (1, 16, 5)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
Completed calculations for domain centre size = 4
Cropped Mask and

In [9]:
glszm_org

array([[[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [4., 0., 0., ..., 0., 0., 0.],
        ...,
        [5., 0., 0., ..., 0., 0., 0.],
        [7., 0., 0., ..., 0., 0., 0.],
        [2., 1., 1., ..., 0., 0., 0.]]])

In [5]:
sae

[0.7776104377993489]

In [6]:
type(sae)

list

In [7]:
sae.astype(str)

AttributeError: 'list' object has no attribute 'astype'

In [46]:
sae

array([0.77761044])

In [47]:
lae

array([9.73098405])

In [48]:
type(lae)

numpy.ndarray

In [49]:
x = np.array([2, 3, 4])

In [50]:
type(x)

numpy.ndarray

In [52]:
type(x[0])


numpy.int32

In [53]:
type(sae[0])

numpy.float64

In [ ]:
type(lae)

In [54]:
x

array([2, 3, 4])

In [ ]:
lae.tolist()

In [55]:
v1

[array([1.]),
 array([1.66666667]),
 array([3.]),
 array([4.64179104]),
 array([7.10169492]),
 array([8.89714286]),
 array([10.47035573]),
 array([12.9893617]),
 array([15.24324324]),
 array([44.15699956]),
 array([119.3203693]),
 array([278.78959652]),
 array([552.12567358]),
 array([979.02152396]),
 array([3199.13638271]),
 array([4345.25479257]),
 array([7414.93971529]),
 array([14351.27118992]),
 array([24648.9535127]),
 array([33640.32507446])]

In [ ]:
np.vstack(v1)

In [ ]:
np.savetxt('test.txt', v1)

In [ ]:
np.savetxt('test2.txt',v2)

In [ ]:
v2

In [56]:
type(v1)

list

In [ ]:
my_df = pd.DataFrame(v1)

In [ ]:
my_df.to_csv('xxxx.csv')

In [19]:
u_window_size.dtye

AttributeError: 'list' object has no attribute 'dtye'

In [11]:
type(u_window_size)

list

In [12]:
v1

[array([1.]),
 array([1.66666667]),
 array([3.]),
 array([4.64179104]),
 array([7.10169492]),
 array([8.89714286]),
 array([10.47035573]),
 array([12.9893617]),
 array([15.24324324]),
 array([44.15699956]),
 array([119.3203693]),
 array([278.78959652]),
 array([552.12567358]),
 array([979.02152396]),
 array([3199.13638271]),
 array([4345.25479257]),
 array([7414.93971529]),
 array([14351.27118992]),
 array([24648.9535127]),
 array([33640.32507446])]

In [22]:
type(sae)

numpy.ndarray

In [23]:
v1[0]

array([1.])

In [24]:
type(v1[0])

numpy.ndarray

In [25]:
sae

array([0.77761044])

In [26]:
sae.tolist()

[0.7776104377993489]

In [27]:
sae.astype(float)

array([0.77761044])

In [28]:
v1[0][0]

1.0

In [29]:
len(v1)

20

In [30]:
len(v1[0])

1

In [31]:
len(u_window_size)

20

In [32]:
len(u_window_size[0])

TypeError: object of type 'int' has no len()

In [33]:
np.sum(ps / (jvector[None, :] ** 2), 1) / Nz

array([0.77761044])

In [34]:
Nz

array([4660363.])

In [35]:
ps

array([[3.414225e+06, 6.893820e+05, 2.334920e+05, 1.073700e+05,
        5.984700e+04, 3.728200e+04, 2.533800e+04, 1.803900e+04,
        1.328600e+04, 1.004900e+04, 7.777000e+03, 6.450000e+03,
        5.183000e+03, 4.106000e+03, 3.393000e+03, 2.955000e+03,
        2.464000e+03, 2.157000e+03, 1.698000e+03, 1.585000e+03,
        1.366000e+03, 1.230000e+03, 1.091000e+03, 1.003000e+03,
        8.330000e+02, 7.550000e+02, 6.040000e+02, 6.270000e+02,
        5.280000e+02, 4.640000e+02, 3.990000e+02, 3.620000e+02,
        3.580000e+02, 3.040000e+02, 3.060000e+02, 2.880000e+02,
        2.320000e+02, 2.100000e+02, 2.070000e+02, 2.090000e+02,
        1.620000e+02, 1.500000e+02, 1.610000e+02, 1.410000e+02,
        1.350000e+02, 1.180000e+02, 1.230000e+02, 1.190000e+02,
        1.020000e+02, 1.000000e+02, 8.300000e+01, 7.900000e+01,
        6.700000e+01, 6.700000e+01, 6.700000e+01, 6.900000e+01,
        5.900000e+01, 5.200000e+01, 6.000000e+01, 5.100000e+01,
        3.400000e+01, 4.700000e+01, 3.00

In [36]:
type(u_window_size[0])

int

In [37]:
type(v1[0])

numpy.ndarray

In [39]:
v1_f = v1[0].astype(float)

In [40]:
v1_f

array([1.])

In [41]:
v1[0]

array([1.])

In [42]:
type(Nz)

numpy.ndarray

In [43]:
type(Nz[0])

numpy.float64